# New Clustering Code 5/10/24


In [ ]:
"""
This is the code for the clusterer - adjusting it here first for testing.
Need to adjust such that each function really only depends upon 1 function above it
Also split up functions as much as possible - want small bits of code
"""
from typing import Any, Dict, Optional

import numpy as np
import sklearn.datasets as dsets
from scipy.cluster.hierarchy import linkage
from scipy.spatial.distance import pdist, squareform

In [ ]:
def get_condensed_distance_array(data):
    return pdist(data, metric="euclidean")


def get_square_distance_matrix(condensed_distance_array):
    return squareform(condensed_distance_array)


def get_linkage_arr(condensed_distance_array):
    return linkage(condensed_distance_array, method="single")


def extend_linkage(link_arr):
    """
    Adds a column to the linkage matrix Z that tracks the new id assigned
    to each row

    Parameters
    ----------
    Z
        linkage matrix

    Returns
    -------
    arr
        linkage matrix with adjusted shape, new shape (Z.shape[0], Z.shape[1]+1)
    """
    # Adjusting linkage matrix to accommodate renumbering
    rows, cols = link_arr.shape
    arr = np.zeros((rows, cols + 1))
    arr[:, :-1] = link_arr
    arr[:, -1] = np.arange(rows + 1, 2 * rows + 1)

    return arr


def get_extended_linkage(condensed_distance_array):
    link_arr = get_linkage_arr(condensed_distance_array)
    return extend_linkage(link_arr)


def fill_missing_cluster_level(left_id, right_id, level, clusters):
    if left_id:
        left_level = left_id[0] + 1
        left_cluster = left_id[1]
        if level != left_level:
            for level_id in range(level - 1, left_level - 2, -1):
                if left_cluster not in clusters[level_id]:
                    clusters[level_id][left_cluster] = {
                        "cluster_merged": False,
                        "count": clusters[left_level - 1][left_cluster]["count"],
                        "avg_dist": clusters[left_level - 1][left_cluster]["avg_dist"],
                        "dist_std": clusters[left_level - 1][left_cluster]["dist_std"],
                        "samples": clusters[left_level - 1][left_cluster]["samples"],
                        "sample_dist": clusters[left_level - 1][left_cluster]["sample_dist"],
                        "outside_1-std": False,
                        "outside_2-std": False,
                    }
    if right_id:
        right_level = right_id[0] + 1
        right_cluster = right_id[1]
        if level != right_level:
            for level_id in range(level - 1, right_level - 2, -1):
                if right_cluster not in clusters[level_id]:
                    clusters[level_id][right_cluster] = {
                        "cluster_merged": False,
                        "count": clusters[right_level - 1][right_cluster]["count"],
                        "avg_dist": clusters[right_level - 1][right_cluster]["avg_dist"],
                        "dist_std": clusters[right_level - 1][right_cluster]["dist_std"],
                        "samples": clusters[right_level - 1][right_cluster]["samples"],
                        "sample_dist": clusters[right_level - 1][right_cluster]["sample_dist"],
                        "outside_1-std": False,
                        "outside_2-std": False,
                    }
    return clusters

In [ ]:
class Clusterer:
    def __init__(self, dataset: np.ndarray, min_num_samples_per_cluster: Optional[int] = None):
        self.min_cluster_size: Optional[int] = min_num_samples_per_cluster
        self._on_init(dataset)

    def _on_init(self, x):
        self._data: np.ndarray = x
        self.num_samples = len(x)
        self.darr: np.ndarray = get_condensed_distance_array(x)
        self.sqdmat: np.ndarray = get_square_distance_matrix(self.darr)
        self.larr: np.ndarray = get_extended_linkage(self.darr)
        self.max_clusters: int = np.count_nonzero(self.larr[:, 3] == 2)
        self.last_merge_level: int = 1
        self.min_num_samples_per_cluster: int = (
            int(min(100, max(2, self.num_samples * 0.05))) if not self.min_cluster_size else self.min_cluster_size
        )

    @property
    def data(self):
        return self._data

    @data.setter
    def data(self, x: np.ndarray):
        self._on_init(x)

    def create_clusters(self) -> Dict[int, Any]:
        """Generates clusters based on linkage matrix

        Returns
        -------
        dict[str, Any]
            Cluster information
        """
        cluster_num = 0
        cluster_tracking = 0
        clusters = {}  # Dictionary to store clusters
        tracking = {}  # Dictionary to associate new cluster ids with actual clusters

        # Walking through the linkage array to generate clusters
        for arr_i in self.larr:
            level = 0

            left_count = 0
            right_count = 0
            merged = False

            arr_0 = int(arr_i[0])  # Grabbing the left id
            arr_1 = int(arr_i[1])  # Grabbing the right id
            dist = arr_i[2]  # Getting the distance between the left and right ids

            new_sample = []
            sample_dist = np.array([dist], dtype=np.float16)

            # Linkage matrix first column id
            left_id = tracking.get(arr_0)  # Determining if the id is already associated with a cluster
            if left_id is None:
                new_sample.append(arr_0)
            else:
                left_cluster = left_id[1]
                left_level = left_id[0] + 1
                left_count = clusters[left_id[0]][left_cluster]["count"]
                left_sample = clusters[left_id[0]][left_cluster]["samples"]
                sample_dist = np.concatenate([clusters[left_id[0]][left_cluster]["sample_dist"], sample_dist])

            # Linkage matrix second column id
            right_id = tracking.get(arr_1)  # Determining if the id is already associated with a cluster
            if right_id is None:
                new_sample.append(arr_1)
            else:
                right_cluster = right_id[1]
                right_level = right_id[0] + 1
                right_count = clusters[right_id[0]][right_cluster]["count"]
                right_sample = clusters[right_id[0]][right_cluster]["samples"]
                sample_dist = np.concatenate([clusters[right_id[0]][right_cluster]["sample_dist"], sample_dist])

            # Aggregate samples, determine cluster number, and get the level
            if left_id and right_id:
                if left_count > right_count:
                    samples = np.concatenate([left_sample, right_sample])
                else:
                    samples = np.concatenate([right_sample, left_sample])
                cluster_num = min([left_cluster, right_cluster])
                merged = max([left_cluster, right_cluster])
                level = max([left_level, right_level])
            elif left_id:
                samples = np.concatenate([left_sample, new_sample])
                cluster_num = left_cluster
                level = left_level
            elif right_id:
                samples = np.concatenate([right_sample, new_sample])
                cluster_num = right_cluster
                level = right_level
            else:
                samples = np.array(new_sample, dtype=np.int32)
                cluster_num = cluster_tracking

            dist_avg = np.mean(sample_dist)
            dist_std = np.std(sample_dist) if sample_dist.shape[0] > 1 else 1e-5

            out1 = dist_avg + dist_std
            out2 = out1 + dist_std

            # Initialize the structure if not present
            if level not in clusters:
                clusters[level] = {
                    cluster_num: {
                        "cluster_merged": merged,
                        "count": samples.shape[0],
                        "avg_dist": dist_avg,
                        "dist_std": dist_std,
                        "samples": samples,
                        "sample_dist": sample_dist,
                        "outside_1-std": dist > out1,
                        "outside_2-std": dist > out2,
                    }
                }
            else:
                clusters[level][cluster_num] = {
                    "cluster_merged": merged,
                    "count": samples.shape[0],
                    "avg_dist": dist_avg,
                    "dist_std": dist_std,
                    "samples": samples,
                    "sample_dist": sample_dist,
                    "outside_1-std": dist > out1,
                    "outside_2-std": dist > out2,
                }

            tracking[int(arr_i[-1])] = (level, cluster_num)  # Associates the new linkage id with the correct cluster

            if not left_id and not right_id:
                # Making sure that new clusters get unique numbers
                cluster_tracking += 1
            else:
                # Fill missing cluster levels for continuity.
                # Ensures all levels have consistent information across cluster changes.
                clusters = fill_missing_cluster_level(left_id, right_id, level, clusters)

            # Only tracking the levels in which clusters merge for the cluster distance matrix
            if merged:
                self.last_merge_level = max(self.last_merge_level, level + 1)

        return clusters

    def get_cluster_distances(self, clusters):
        # this is the cluster distance matrix
        cluster_matrix = np.full((self.last_merge_level, self.max_clusters, self.max_clusters), -1.0, dtype=np.float32)

        for level, cluster_set in clusters.items():
            if level < self.last_merge_level:
                cluster_ids = sorted(cluster_set.keys())
                for i, cluster_id in enumerate(cluster_ids):
                    cluster_matrix[level, cluster_id, cluster_id] = clusters[level][cluster_id]["avg_dist"]
                    for int_id in range(i + 1, len(cluster_ids)):
                        compare_id = cluster_ids[int_id]
                        sample_a = clusters[level][cluster_id]["samples"]
                        sample_b = clusters[level][compare_id]["samples"]
                        min_mat = self.sqdmat[np.ix_(sample_a, sample_b)].min()
                        cluster_matrix[level, cluster_id, compare_id] = min_mat
                        cluster_matrix[level, compare_id, cluster_id] = min_mat

        return cluster_matrix

    def get_merge_levels(self, clusters):
        """
        Runs through the clusters dictionary determining when clusters merge,
        and how close are those clusters when they merge.

        Parameters
        ----------
        clusters:
            A dictionary containing the original clusters information.

        Returns
        -------
        merge_clusters:
            A dictionary with each clusters merge history
        """

        merge_clusters = {"merge": {}, "likely_merge": {}, "no_merge": {}}

        for level, cluster_set in clusters.items():
            cluster_ids = sorted(cluster_set.keys())
            for i, cluster_id in enumerate(cluster_ids):
                # Extract necessary information
                samples = clusters[level][cluster_id]["samples"]
                merged = clusters[level][cluster_id]["cluster_merged"]
                out1 = clusters[level][cluster_id]["outside_1-std"]
                out2 = clusters[level][cluster_id]["outside_2-std"]

                if merged:
                    if out2:
                        if len(samples) < self.min_num_samples_per_cluster:
                            if cluster_id not in merge_clusters["likely_merge"]:
                                merge_clusters["likely_merge"][cluster_id] = {level: [merged, "low"]}
                            if level not in merge_clusters["likely_merge"][cluster_id]:
                                merge_clusters["likely_merge"][cluster_id][level] = [merged, "low"]
                        else:
                            if cluster_id not in merge_clusters["no_merge"]:
                                merge_clusters["no_merge"][cluster_id] = {level: [merged]}
                            if level not in merge_clusters["no_merge"][cluster_id]:
                                merge_clusters["no_merge"][cluster_id][level] = [merged]

                    elif out1 and len(samples) >= self.min_num_samples_per_cluster:
                        if cluster_id not in merge_clusters["likely_merge"]:
                            merge_clusters["likely_merge"][cluster_id] = {level: [merged]}
                        if level not in merge_clusters["likely_merge"][cluster_id]:
                            merge_clusters["likely_merge"][cluster_id][level] = [merged]

                    else:
                        if cluster_id not in merge_clusters["merge"]:
                            merge_clusters["merge"][cluster_id] = {level: [merged]}
                        if level not in merge_clusters["merge"][cluster_id]:
                            merge_clusters["merge"][cluster_id][level] = [merged]

        return merge_clusters

    def generate_merge_list(self, cluster_merges, cluster_matrix):
        merge_list, merge_mean, intra_max, additional_check = self.cluster_merging(cluster_merges, cluster_matrix)
        desired_merge, merge = self.get_desired_merge(merge_mean, intra_max, additional_check)

        j = 0
        for i, select in enumerate(desired_merge):
            if select:
                merge_list[i].append("merge")
            else:
                if merge[j]:
                    merge_list[i].append("merge")
                else:
                    merge_list[i].append("no-merge")
                j += 1

        merge_list = sorted(merge_list, reverse=True)
        return merge_list

    def cluster_merging(self, cluster_merges, cluster_matrix):
        intra_max = []
        merge_mean = []
        merge_list = []
        additional_check = []
        # Process each merge type
        for merge_type, merge_clusters in cluster_merges.items():
            for outer_cluster, inner_clusters in merge_clusters.items():
                for level, cluster_list in inner_clusters.items():
                    # Determine if there is a small far merge
                    if len(cluster_list) == 2:
                        additional_check.append(True)
                    else:
                        additional_check.append(False)
                    inner_cluster = cluster_list[0]

                    # Get the slice of the distance matrix up to the level before merging
                    distances = cluster_matrix[:level, outer_cluster, inner_cluster]
                    # print(f"Negative check for {outer_cluster}-{inner_cluster} at {level} : {np.any(distances<0)}")
                    # if np.any(distances < 0):
                    #     print(distances)
                    intra_distance = cluster_matrix[:, outer_cluster, outer_cluster]
                    mask = intra_distance >= 0
                    intra_filtered = intra_distance[mask]
                    intra_max.append(np.max(intra_filtered))

                    # Grabbing the corresponding desired values
                    if merge_type == "merge":
                        merge_mean.append(np.max(distances))
                    else:
                        merge_mean.append(np.mean(distances))

                    merge_list.append([level, outer_cluster, inner_cluster])

        return merge_list, merge_mean, intra_max, additional_check

    def get_desired_merge(self, merge_mean, intra_max, additional_check):
        intra_max = np.unique(intra_max)
        intra_value = np.log(intra_max)
        intra_value = intra_value.mean() + 2 * intra_value.std()
        merge_value = np.log(merge_mean)
        desired_merge = merge_value < intra_value

        check = merge_value[~desired_merge]
        check = np.abs((check - intra_value) / intra_value)
        mask = check < 1
        good = check[mask].mean() + check[mask].std()
        merge = check < good
        return desired_merge, merge

    def get_last_merge_levels(self, merge_list):
        last_good_merge_levels = {}
        for entry in merge_list:
            level, outer_cluster, inner_cluster, status = entry
            if status == "no-merge":
                if outer_cluster not in last_good_merge_levels:
                    last_good_merge_levels[outer_cluster] = 1
                if inner_cluster not in last_good_merge_levels:
                    last_good_merge_levels[inner_cluster] = 1
                if last_good_merge_levels[outer_cluster] > level:
                    last_good_merge_levels[outer_cluster] = level - 1
            else:
                if outer_cluster in last_good_merge_levels:
                    last_good_merge_levels[outer_cluster] = max(last_good_merge_levels[outer_cluster], level)
        return last_good_merge_levels

    def fill_dedup_std_list(self, last_merge_levels):
        pass

    def find_duplicates(self, dedup_std_list):
        diag_mask = np.ones(self.sqdmat.shape, dtype=bool)
        np.fill_diagonal(diag_mask, 0)
        diag_mask = np.triu(diag_mask)

        exact_mask = self.sqdmat < (np.mean(dedup_std_list) / 100)
        exact_indices = np.nonzero(exact_mask & diag_mask)
        exact_dedup = list(zip(exact_indices[0], exact_indices[1]))

        possible_mask = self.sqdmat < np.mean(dedup_std_list)
        possible_indices = np.nonzero(possible_mask & diag_mask & ~exact_mask)
        possible_dedup = list(zip(possible_indices[0], possible_indices[1]))

        return exact_dedup, possible_dedup

    def find_outliers(self, clusters, last_merge_levels):
        """
        The clusters dictionary contains whether the added sample/cluster
        was outside 1 standard deviation or outside 2 standard deviations.
        last_merge_levels contains the last good merge for each cluster we care about
        Using this information to determine when the sample was added to the cluster
        and how far it was from the cluster when it was added

        """
        outliers = []
        possible_outliers = []

        for level, cluster_set in clusters.items():
            cluster_ids = sorted(cluster_set.keys())
            for i, cluster_id in enumerate(cluster_ids):
                # Extract necessary information
                samples = clusters[level][cluster_id]["samples"]
                merged = clusters[level][cluster_id]["cluster_merged"]
                out1 = clusters[level][cluster_id]["outside_1-std"]
                out2 = clusters[level][cluster_id]["outside_2-std"]

                if cluster_id in last_merge_levels and not merged:
                    if level > last_merge_levels[cluster_id] and out2:
                        outliers.append(samples[-1])
                    elif (
                        level > last_merge_levels[cluster_id]
                        and out1
                        and len(samples) >= self.min_num_samples_per_cluster
                    ):
                        possible_outliers.append(samples[-1])
        return outliers, possible_outliers

    def run(self):
        sample_info = self.create_clusters()

        print("Clusters per level:")
        for level, stuff in sample_info.items():
            print(f"\t{level}")
            for cluster, more_stuff in stuff.items():
                print(f"\t\t{cluster} - {more_stuff['cluster_merged']}")

        if self.max_clusters > 1:
            cluster_matrix = self.get_cluster_distances(sample_info)
            merge_levels = self.get_merge_levels(sample_info)
            merge_list = self.generate_merge_list(merge_levels, cluster_matrix)
            last_merge_levels = self.get_last_merge_levels(merge_list)
        else:
            last_merge_levels = {0: int(max(self.num_samples * 0.1, self.min_num_samples_per_cluster))}

        outliers, potential_outliers = self.find_outliers(sample_info, last_merge_levels)

        dedup_std = []
        for cluster, level in last_merge_levels.items():
            level_cluster = sample_info[level][cluster]
            samples = level_cluster["samples"]
            if samples.shape[0] < self.min_num_samples_per_cluster:
                outliers.extend(samples.tolist())
            else:
                dedup_std.append(level_cluster["dist_std"])

        duplicates, near_duplicates = self.find_duplicates(dedup_std)

        ret = {
            "outliers": outliers,
            "potential_outliers": potential_outliers,
            "duplicates": duplicates,
            "near_duplicates": near_duplicates,
        }

        return ret

In [ ]:
class Clusterer2:
    def __init__(self, dataset: np.ndarray):
        """ """
        # This is done to update the state rather than instantiate a new class when new data is passed in
        self._on_init(dataset)

    def _on_init(self, dataset: np.ndarray):
        self._data: np.ndarray = dataset
        self.num_samples = len(dataset)
        self.darr: np.ndarray = get_condensed_distance_array(dataset)
        self.sqdmat: np.ndarray = get_square_distance_matrix(self.darr)
        self.larr: np.ndarray = get_extended_linkage(self.darr)
        self.max_clusters: int = np.count_nonzero(self.larr[:, 3] == 2)
        self.last_merge_level: int = 1

        min_num = int(self.num_samples * 0.05)
        self.min_num_samples_per_cluster = clamp(min_num, 2, 100)

    @property
    def data(self):
        return self._data

    @data.setter
    def data(self, x: np.ndarray):
        self._on_init(x)

    def create_clusters(self) -> Dict[int, Any]:
        """Generates clusters based on linkage matrix

        Returns
        -------
        dict[str, Any]
            Cluster information
        """
        cluster_num = 0
        cluster_tracking = 0
        clusters = {}  # Dictionary to store clusters
        tracking = {}  # Dictionary to associate new cluster ids with actual clusters

        # Walking through the linkage array to generate clusters
        for arr_i in self.larr:
            level = 0
            merged = False

            arr_0 = int(arr_i[0])  # Grabbing the left id
            arr_1 = int(arr_i[1])  # Grabbing the right id
            dist = arr_i[2]
            sample_dist = np.array([dist], dtype=np.float16)

            # Linkage matrix first column id
            left_id = tracking.get(arr_0)  # Determining if the id is already associated with a cluster
            if left_id:
                left_cluster = left_id[1]
                left_level = left_id[0] + 1
                left_sample = clusters[left_id[0]][left_cluster]["samples"]
                sample_dist = np.concatenate([clusters[left_id[0]][left_cluster]["sample_dist"], sample_dist])
            # Linkage matrix second column id
            right_id = tracking.get(arr_1)  # Determining if the id is already associated with a cluster
            if right_id:
                right_cluster = right_id[1]
                right_level = right_id[0] + 1
                right_sample = clusters[right_id[0]][right_cluster]["samples"]
                sample_dist = np.concatenate([clusters[right_id[0]][right_cluster]["sample_dist"], sample_dist])

            # Aggregate samples, determine cluster number, and get the level
            if left_id and right_id:
                if clusters[left_id[0]][left_cluster]["count"] > clusters[right_id[0]][right_cluster]["count"]:
                    samples = np.concatenate([left_sample, right_sample])
                else:
                    samples = np.concatenate([right_sample, left_sample])
                cluster_num = min([left_cluster, right_cluster])
                merged = max([left_cluster, right_cluster])
                level = max([left_level, right_level])
                # Only tracking the levels in which clusters merge for the cluster distance matrix
                self.last_merge_level = max(self.last_merge_level, level + 1)
            elif left_id:
                samples = np.concatenate([left_sample, [arr_1]])
                cluster_num = left_cluster
                level = left_level
            elif right_id:
                samples = np.concatenate([right_sample, [arr_0]])
                cluster_num = right_cluster
                level = right_level
            else:
                samples = np.array([arr_0, arr_1], dtype=np.int32)
                cluster_num = cluster_tracking

            # Calculate distances
            dist_avg = np.mean(sample_dist)
            dist_std = np.std(sample_dist) if sample_dist.shape[0] > 1 else 1e-5

            out1 = dist_avg + dist_std
            out2 = out1 + dist_std

            # Initialize the structure if not present
            if level not in clusters:
                clusters[level] = {}

            clusters[level][cluster_num] = {
                "cluster_merged": merged,
                "count": samples.shape[0],
                "avg_dist": dist_avg,
                "dist_std": dist_std,
                "samples": samples,
                "sample_dist": sample_dist,
                "outside_1-std": dist > out1,
                "outside_2-std": dist > out2,
            }

            tracking[int(arr_i[-1])] = (level, cluster_num)  # Associates the new linkage id with the correct cluster

            # If no left or right id, increment tracker to ensure new clusters get unique ids
            if not left_id and not right_id:
                cluster_tracking += 1
            # Update clusters to include previously skipped levels
            if left_id and left_id[0] + 1 != level:
                clusters = self.fill_level(left_id, level, clusters)
            if right_id and right_id[0] + 1 != level:
                clusters = self.fill_level(right_id, level, clusters)

        return clusters

    def fill_level(self, cluster_id, level, clusters):
        new_level, cluster = cluster_id[0] + 1, cluster_id[1]
        cluster_info = {
            "cluster_merged": False,
            "count": clusters[new_level - 1][cluster]["count"],
            "avg_dist": clusters[new_level - 1][cluster]["avg_dist"],
            "dist_std": clusters[new_level - 1][cluster]["dist_std"],
            "samples": clusters[new_level - 1][cluster]["samples"],
            "sample_dist": clusters[new_level - 1][cluster]["sample_dist"],
            "outside_1-std": False,
            "outside_2-std": False,
        }
        # Sets each level's cluster info if it does not exist
        for level_id in range(level - 1, new_level - 2, -1):
            clusters[level_id].setdefault(cluster, cluster_info)

        return clusters

    def get_cluster_distances(self, clusters):
        # this is the cluster distance matrix
        cluster_matrix = np.full((self.last_merge_level, self.max_clusters, self.max_clusters), -1.0, dtype=np.float32)

        for level, cluster_set in clusters.items():
            if level < self.last_merge_level:
                cluster_ids = sorted(cluster_set.keys())
                for i, cluster_id in enumerate(cluster_ids):
                    cluster_matrix[level, cluster_id, cluster_id] = clusters[level][cluster_id]["avg_dist"]
                    for int_id in range(i + 1, len(cluster_ids)):
                        compare_id = cluster_ids[int_id]
                        sample_a = clusters[level][cluster_id]["samples"]
                        sample_b = clusters[level][compare_id]["samples"]
                        min_mat = self.sqdmat[np.ix_(sample_a, sample_b)].min()
                        cluster_matrix[level, cluster_id, compare_id] = min_mat
                        cluster_matrix[level, compare_id, cluster_id] = min_mat

        return cluster_matrix

    def calc_merge_indices(self, merge_mean, intra_max):
        intra_max_uniques = np.unique(intra_max)
        intra_log_values = np.log(intra_max_uniques)
        two_std_all = intra_log_values.mean() + 2 * intra_log_values.std()
        merge_value = np.log(merge_mean)
        # Mask of indices we know we want to merge
        desired_merge = merge_value < two_std_all

        # List[Values] for indices we might want to merge
        check = merge_value[~desired_merge]
        # Check distance from value to 2 stds of all values
        check = np.abs((check - two_std_all) / two_std_all)
        # Mask List[Values < 1]
        mask = check < 1
        one_std_check = check[mask].mean() + check[mask].std()
        # Mask of indices that should also be merged
        mask2_vals = np.abs((merge_value - two_std_all) / two_std_all)
        mask2 = mask2_vals < one_std_check
        return np.logical_or(desired_merge, mask2)

    def generate_merge_list(self, clusters: dict, cluster_matrix: np.ndarray):
        """
        Runs through the clusters dictionary determining when clusters merge,
        and how close are those clusters when they merge.

        Parameters
        ----------
        clusters:
            A dictionary containing the original clusters information.

        Returns
        -------
        merge_clusters:
            A dictionary with each clusters merge history
        """
        intra_max = []
        merge_mean = []
        merge_list = []

        for level, cluster_set in clusters.items():
            for outer_cluster, cluster_info in cluster_set.items():
                inner_cluster = cluster_info["cluster_merged"]
                if not inner_cluster:
                    continue
                # Extract necessary information
                num_samples = len(cluster_info["samples"])
                out1 = cluster_info["outside_1-std"]
                out2 = cluster_info["outside_2-std"]

                # If outside 2-std or 1-std and larger than a minimum sized cluster, take the mean distance, else max
                aggregate_func = (
                    np.mean if out2 or (out1 and num_samples >= self.min_num_samples_per_cluster) else np.max
                )

                distances = cluster_matrix[:level, outer_cluster, inner_cluster]
                intra_distance = cluster_matrix[:, outer_cluster, outer_cluster]
                positive_mask = intra_distance >= 0
                intra_filtered = intra_distance[positive_mask]

                # TODO: Append now, take max over axis later?
                intra_max.append(np.max(intra_filtered))
                # Calculate the corresponding distance stats
                distance_stats_arr = aggregate_func(distances)
                merge_mean.append(distance_stats_arr)
                merge_list.append([level, outer_cluster, inner_cluster])

        all_merge_indices = self.calc_merge_indices(merge_mean=merge_mean, intra_max=intra_max)

        for i, is_mergeable in enumerate(all_merge_indices):
            merge_list[i].append(is_mergeable)

        merge_list = sorted(merge_list, reverse=True)

        return merge_list

    def get_last_merge_levels(self, merge_list):
        last_good_merge_levels = {}
        for entry in merge_list:
            level, outer_cluster, inner_cluster, status = entry
            if not status:
                if outer_cluster not in last_good_merge_levels:
                    last_good_merge_levels[outer_cluster] = 1
                if inner_cluster not in last_good_merge_levels:
                    last_good_merge_levels[inner_cluster] = 1
                if last_good_merge_levels[outer_cluster] > level:
                    last_good_merge_levels[outer_cluster] = level - 1
            else:
                if outer_cluster in last_good_merge_levels:
                    last_good_merge_levels[outer_cluster] = max(last_good_merge_levels[outer_cluster], level)
        return last_good_merge_levels

    def find_outliers(self, clusters, last_merge_levels):
        """
        The clusters dictionary contains whether the added sample/cluster
        was outside 1 standard deviation or outside 2 standard deviations.
        last_merge_levels contains the last good merge for each cluster we care about
        Using this information to determine when the sample was added to the cluster
        and how far it was from the cluster when it was added

        """
        outliers = []
        possible_outliers = []

        for level, cluster_set in clusters.items():
            # cluster_ids = sorted(cluster_set.keys())
            for cluster_id, cluster_info in cluster_set.items():
                # Extract necessary information
                samples = cluster_info["samples"]
                merged = cluster_info["cluster_merged"]
                out1 = cluster_info["outside_1-std"]
                out2 = cluster_info["outside_2-std"]

                if cluster_id in last_merge_levels and not merged:
                    if level > last_merge_levels[cluster_id] and out2:
                        outliers.append(samples[-1])
                    elif (
                        level > last_merge_levels[cluster_id]
                        and out1
                        and len(samples) >= self.min_num_samples_per_cluster
                    ):
                        possible_outliers.append(samples[-1])
        return outliers, possible_outliers

    def calc_duplicate_std(self, info, last_merge_levels):
        dedup_std = []
        additional_outliers = []
        for cluster, level in last_merge_levels.items():
            samples = info[level][cluster]["samples"]
            if samples.shape[0] < self.min_num_samples_per_cluster:
                additional_outliers.extend(samples.tolist())
            else:
                dedup_std.append(info[level][cluster]["dist_std"])
        return dedup_std, additional_outliers

    def find_duplicates(self, dedup_std_list):
        diag_mask = np.ones_like(self.sqdmat, dtype=bool)
        np.fill_diagonal(diag_mask, 0)
        diag_mask = np.triu(diag_mask)

        exact_mask = self.sqdmat < (np.mean(dedup_std_list) / 100)
        exact_indices = np.nonzero(exact_mask & diag_mask)
        exact_dedup = list(zip(exact_indices[0], exact_indices[1]))

        possible_mask = self.sqdmat < np.mean(dedup_std_list)
        possible_indices = np.nonzero(possible_mask & diag_mask & ~exact_mask)
        possible_dedup = list(zip(possible_indices[0], possible_indices[1]))

        return exact_dedup, possible_dedup

    def run(self):
        clusters_info_per_level = self.create_clusters()

        if self.max_clusters > 1:
            cluster_matrix = self.get_cluster_distances(clusters_info_per_level)
            merge_list = self.generate_merge_list(clusters_info_per_level, cluster_matrix)
            last_good_merge_levels = self.get_last_merge_levels(merge_list)
        else:
            last_good_merge_levels = {0: int(self.num_samples * 0.1)}

        outliers, potential_outliers = self.find_outliers(clusters_info_per_level, last_good_merge_levels)
        # duplicates_std, additional_outliers = self.calc_duplicate_std(clusters_info_per_level, last_good_merge_levels)

        duplicates_std = []
        additional_outliers = []
        for cluster, level in last_good_merge_levels.items():
            samples = clusters_info_per_level[level][cluster]["samples"]
            if len(samples) < self.min_num_samples_per_cluster:
                additional_outliers.extend(samples.tolist())
            else:
                duplicates_std.append(clusters_info_per_level[level][cluster]["dist_std"])

        outliers.extend(additional_outliers)
        duplicates, near_duplicates = self.find_duplicates(duplicates_std)

        ret = {
            "outliers": outliers,
            "potential_outliers": potential_outliers,
            "duplicates": duplicates,
            "near_duplicates": near_duplicates,
        }

        return ret

# Testing Data


In [ ]:
plot_kwds = {"alpha": 0.5, "s": 50, "linewidths": 0}

# moons, _ = dsets.make_moons(n_samples=50, noise=0.1)
blobs, _ = dsets.make_blobs(  # type: ignore
    n_samples=100,
    centers=[(-1.5, 1.8), (-1, 3), (0.8, 2.1), (2.8, 1.5), (2.5, 3.5)],  # type: ignore
    cluster_std=0.3,
    random_state=33,
)
# test_data = np.vstack([moons, blobs])
test_data = blobs
test_data[79] = test_data[24]
test_data[63] = test_data[58] + 1e-5

# Test Functions


In [ ]:
# Shaun's code for testing the clusterer
def test_outliers(x):
    assert len(x) == 6
    for val in x:
        assert val in [21, 6, 4, 71, 38, 11]
    print("Passed")


def test_potential_outliers(x):
    assert len(x) == 5
    for val in x:
        assert val in [42, 48, 9, 1, 43]
    print("Passed")


def test_duplicates(x):
    assert x == [(24, 79), (58, 63)]
    print("Passed")


def test_near_duplicates(x):
    assert x == [
        (8, 27),
        (10, 65),
        (16, 99),
        (19, 64),
        (22, 87),
        (27, 29),
        (33, 76),
        (39, 55),
        (40, 72),
        (41, 62),
        (80, 81),
        (80, 93),
        (81, 93),
        (87, 95),
    ]
    print("Passed")


def _recursive_dict_equality(d1: dict, d2: dict):
    for k, v in d1.items():
        if isinstance(v, dict):
            _recursive_dict_equality(v, d2[k])
        else:
            x = np.array([v]).flatten()
            y = np.array([d2[k]]).flatten()
            assert all(x == y)


def test_recursive_dicts(d1: dict, d2: dict):
    _recursive_dict_equality(d1, d2)
    print("Dict Equality: Passed")


def _recursive_list_equality(l1: list, l2: list):
    assert len(l1) == len(l2)
    for i1, i2 in zip(l1, l2):
        if isinstance(i1, list):
            _recursive_list_equality(i1, i2)
        else:
            assert i1 == i2


def test_lists_equal(l1: list, l2: list):
    _recursive_list_equality(l1, l2)
    print("List Equality: Passed")


def test_matrix_equal(m1: np.ndarray, m2: np.ndarray):
    np.testing.assert_array_equal(m1, m2)
    print("Matrix Equality: Passed")


def run_tests(x):
    test_outliers(x["outliers"])
    test_potential_outliers(x["potential_outliers"])
    test_duplicates(x["duplicates"])
    test_near_duplicates(x["near_duplicates"])

# Run Tests


In [ ]:
from importlib import reload

import daml._internal.metrics.clustering as clustering

reload(clustering)

coo = clustering.Clusterer(test_data)
coo.create_clusters()
coo.run()

In [ ]:
from clusterer_new import Clusterer as Clusterer2

clusterer = Clusterer(test_data)
clusterer2 = Clusterer2(test_data)
info = clusterer.create_clusters()
info2 = clusterer2.create_clusters()
test_recursive_dicts(info, info2)

cluster_matrix = clusterer.get_cluster_distances(info)
cluster_matrix2 = clusterer2.get_cluster_distances(info2)
test_matrix_equal(cluster_matrix, cluster_matrix2)

merge_levels = clusterer.get_merge_levels(info)
# merge_levels2 = clusterer2.get_merge_levels(info2)
# test_recursive_dicts(merge_levels, merge_levels2)

merge_list = clusterer.generate_merge_list(merge_levels, cluster_matrix)
merge_list2 = clusterer2.generate_merge_list(info2, cluster_matrix2)
# Replaced test since "status" is a str vs bool for the two versions
assert len(merge_list2) == len(merge_list)
for l1, l2 in zip(merge_list, merge_list2):
    assert l1[:3] == l2[:3]
    assert (l1[-1] == "merge") == l2[-1]
# test_lists_equal(merge_list, merge_list2)

last_merge_levels = clusterer.get_last_merge_levels(merge_list)
last_merge_levels2 = clusterer2.get_last_merge_levels(merge_list2)
test_recursive_dicts(last_merge_levels, last_merge_levels2)

outliers, potential_outliers = clusterer.find_outliers(info, last_merge_levels)
outliers2, potential_outliers2 = clusterer2.find_outliers(info, last_merge_levels2)
test_lists_equal(sorted(outliers), sorted(outliers2))
test_lists_equal(potential_outliers, potential_outliers2)

dedup_std = []
for cluster, level in last_merge_levels.items():
    level_cluster = info[level][cluster]
    samples = level_cluster["samples"]
    if samples.shape[0] < clusterer.min_num_samples_per_cluster:
        outliers.extend(samples.tolist())
    else:
        dedup_std.append(level_cluster["dist_std"])

dedup_std2 = []
additional_outliers = []
for cluster, level in last_merge_levels2.items():
    samples = info[level][cluster]["samples"]
    if samples.shape[0] < clusterer2.min_num_samples_per_cluster:
        additional_outliers.extend(samples.tolist())
    else:
        dedup_std2.append(info[level][cluster]["dist_std"])

duplicates, near_duplicates = clusterer.find_duplicates(dedup_std)
# dedup_std2 = clusterer2.calc_du
duplicates2, near_duplicates2 = clusterer2.find_duplicates(dedup_std2)
test_lists_equal(duplicates, duplicates2)
test_lists_equal(near_duplicates, near_duplicates2)

### Timing Original


In [ ]:
print(">>>>>\tCreate Clusters\t<<<<<")
%timeit clusterer.create_clusters()
%timeit clusterer2.create_clusters()
print(">>>>>\tCluster Distances\t<<<<<")
%timeit clusterer.get_cluster_distances(info)
%timeit clusterer2.get_cluster_distances(info2)
print(">>>>>\tMerge Levels\t<<<<<")
%timeit clusterer.get_merge_levels(info)
# %timeit clusterer2.get_merge_levels(info2)
print(">>>>>\tMerge List\t<<<<<")
%timeit clusterer.generate_merge_list(merge_levels, cluster_matrix)
%timeit clusterer2.generate_merge_list(info2, cluster_matrix2)
print(">>>>>\tLast Merge Levels\t<<<<<")
%timeit clusterer.get_last_merge_levels(merge_list)
%timeit clusterer2.get_last_merge_levels(merge_list2)
print(">>>>>\tOutliers\t<<<<<")
%timeit clusterer.find_outliers(info, last_merge_levels)
%timeit clusterer2.find_outliers(info2, last_merge_levels2)
print(">>>>>\tDuplicates\t<<<<<")
%timeit clusterer.find_duplicates(dedup_std)
%timeit clusterer2.find_duplicates(dedup_std)

### Timing Changes


In [ ]:
%%timeit
clusterer.create_clusters()
clusterer.get_cluster_distances(info)
clusterer.get_merge_levels(info)
clusterer.generate_merge_list(merge_levels, cluster_matrix2)
clusterer.get_last_merge_levels(merge_list)
clusterer.find_outliers(info, last_merge_levels2)
clusterer.find_duplicates(dedup_std)

In [ ]:
%%timeit
clusterer2.create_clusters()
clusterer2.get_cluster_distances(info2)
clusterer2.generate_merge_list(info2, cluster_matrix2)
clusterer2.get_last_merge_levels(merge_list2)
clusterer2.find_outliers(info2, last_merge_levels2)
clusterer2.find_duplicates(dedup_std)